In [1]:
import os
import pandas as pd

infile = '../../HGT_demo_file/SAMEA3449210.event_output.csv'
db_idir = '../../HGT_demo_file/DB.genome_annotation'
outdir = '.'
fr_size = 1000

df = pd.read_csv(infile, header=0, index_col=None)
# db = pd.read_csv(db_file, header=0, index_col=0, sep='\t')

df.rename(columns={'receptor':'recipient'}, inplace=True)

In [2]:
def overlap(range1, range2):
    if range1[0] > range2[1] or range1[1] < range2[0]:
        return False
    else:
        return True

# styp = recipient or donor
def search_event(scaffold, db, range):
    scaffold_df = db[db['Chr'] == scaffold]
    valid_idx = []
    for idx in scaffold_df.index:
        if overlap(range, [scaffold_df.loc[idx, 'Start'], scaffold_df.loc[idx, 'End']]):
            valid_idx.append(idx)
    valid_df = scaffold_df.loc[valid_idx, ]
    return valid_df
    

def search_row(idx, df, db_dir, fr_size):
    tmp = '{}.ImmeDB.tsv'
    row = df.loc[idx, ]
    # for recipient
    recipient = row['recipient']
    chrom = recipient.split('_')[0]
    ifile = os.path.join(db_dir, tmp.format(chrom))
    db = pd.read_csv(ifile, header=0, index_col=None, sep='\t')
    recipient_range = [max(0, row['insert_locus']-fr_size), row['insert_locus']+fr_size]
    recipient_df = search_event(recipient, db, recipient_range)
    # for donor
    donor = row['donor']
    range = [max(0, row['delete_start'] - fr_size), row['delete_end'] + fr_size]
    chrom = donor.split('_')[0]
    ifile = os.path.join(db_dir, tmp.format(chrom))
    db = pd.read_csv(ifile, header=0, index_col=None, sep='\t')
    donor_df = search_event(donor, db, range)
    return recipient_df, donor_df 



In [3]:
result_anno = pd.DataFrame(columns=['id', 'sample', 'recipient_MGE_n', 'recipient_MGE_category', 'recipient_MGE_list', 'donor_MGE_n', 'donor_MGE_category', 'donor_MGE_list', 'recipient', 'insert_locus', 'donor', 'delete_start', 'delete_end', 'reverse_flag'])
for idx in df.index:
    recipient_df, donor_df = search_row(idx, df, db_idir, fr_size)
    id = 'HGT_c{}'.format(idx+1)
    sample = df.loc[idx, 'sample']
    recipient_MGE_n = recipient_df.shape[0]
    recipient_MGE_category = ';'.join(recipient_df['Category'])
    recipient_MGE_list = ';'.join(recipient_df['Name'])
    if recipient_MGE_n == 0:
        recipient_MGE_list = 'NA'
        recipient_MGE_category = 'NA'
    donor_MGE_n = donor_df.shape[0]
    donor_MGE_category = ';'.join(recipient_df['Category'])
    donor_MGE_list = ';'.join(recipient_df['Name'])
    if donor_MGE_n == 0:
        donor_MGE_list = 'NA'
        donor_MGE_category = 'NA'
    recipient = df.loc[idx, 'recipient']
    insert_locus = df.loc[idx, 'insert_locus']
    donor = df.loc[idx, 'donor']
    delete_start = df.loc[idx, 'delete_start']
    delete_end = df.loc[idx, 'delete_end']
    reverse_flag = df.loc[idx, 'reverse_flag']
    result_anno.loc[len(result_anno), ] = [id, sample, recipient_MGE_n, recipient_MGE_category, recipient_MGE_list, donor_MGE_n, donor_MGE_category, donor_MGE_list, recipient, insert_locus, donor, delete_start, delete_end, reverse_flag]
    #result_anno.iloc[len(result_anno), ] = [id, sample, recipient_HGTC_n, recipient_HGTC_list, donor_HGTC_n, donor_HGTC_list, recipient, insert_locus, donor, delete_start, delete_end, reverse_flag]

merge_df = pd.concat([recipient_df, donor_df], ignore_index=True)
merge_df.to_csv(os.path.join(outdir, 'output.MGE_annotation.MGE.tsv '), index=False, sep='\t')
result_anno.to_csv(os.path.join(outdir, 'output.MGE_annotation.annotated.tsv'), index=False, sep='\t')


In [4]:
result_anno

,id,sample,recipient_MGE_n,recipient_MGE_category,recipient_MGE_list,donor_MGE_n,donor_MGE_category,donor_MGE_list,recipient,insert_locus,donor,delete_start,delete_end,reverse_flag
0,HGT_c1,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000002926_8,51904,MGYG000000070_3,80170,81829,True
1,HGT_c2,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000001132_27,72500,MGYG000004551_1,33911,35369,True
2,HGT_c3,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000001338_2,16000,MGYG000004481_60,119,1814,True
3,HGT_c4,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000000003_1,491015,MGYG000001420_2,1240944,1256164,True
4,HGT_c5,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000001940_46,794,MGYG000002661_58,4674,5482,False
5,HGT_c6,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000000003_6,93988,MGYG000003084_128,574,1931,False
6,HGT_c7,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000002143_1,89223,MGYG000002156_4,72097,80022,False
7,HGT_c8,SAMEA3449210,0,NA,NA,1,,,MGYG000002673_33,1879,MGYG000001698_6,17050,25240,True
8,HGT_c9,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000002416_12,54320,MGYG000003004_57,536,1349,True
9,HGT_c10,SAMEA3449210,0,NA,NA,0,NA,NA,MGYG000002454_1,1391624,MGYG000001921_7,61159,91866,True
